In [1]:
import numpy as np
from qore import Mine, ASP, QAOA, VQE
from qore.algorithms.pseudoflow import Pseudoflow
from qore.utils import measure_operator, get_bitstring_probabilities

In [2]:
penalty = 10.0
qmine = Mine(np.array([[-2., 3., 1.], [float('inf'), 5., float('inf')]]))
qmine.plot_mine()

+---+--------+-------+-------+
|   |   0    |   1   |   2   |
+---+--------+-------+-------+
| 0 | -2.000 | 3.000 | 1.000 |
| 1 |  inf   | 5.000 |  inf  |
+---+--------+-------+-------+
/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/quantum_info/operators/symplectic/pauli.py:185: DeprecationWarning: Initializing Pauli from `Pauli(z=z, x=x)` kwargs is deprecated as of version 0.17.0 and will be removed no earlier than 3 months after the release date. Use tuple initialization `Pauli((z, x))` instead.
  base_z, base_x, base_phase = self._from_array_deprecated(z, x)


In [3]:
graph, source, sink = qmine.gen_pseudoflow_graph()

In [4]:
pf = Pseudoflow(graph, source, sink)

In [5]:
pf.run()

{'ground_state': '1111'}

In [6]:
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit import Aer

In [7]:
aqua_globals.random_seed = 1953
backend = Aer.get_backend('statevector_simulator')
instance = QuantumInstance(backend=backend)

<ipython-input-7-305a6dd83b3e>:1: DeprecationWarning: The variable qiskit.aqua.aqua_globals is deprecated. It was moved/refactored to qiskit.utils.aqua_globals (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  aqua_globals.random_seed = 1953
/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/aqua/quantum_instance.py:135: DeprecationWarning: The class qiskit.aqua.QuantumInstance is deprecated. It was moved/refactored to qiskit.utils.QuantumInstance (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_class('aqua.QuantumInstance',


In [8]:
def analysis(circuit):
    print("Average Profit:                          ", measure_operator(qmine.Hp, circuit, instance))
    print("Average Number of Smoothness Violations: ", measure_operator(qmine.Hs, circuit, instance) / penalty)
    x = get_bitstring_probabilities(circuit, instance, shots=1024)
    bitstr, prob = max(x.items(), key=lambda item: item[1])
    print(f"The most probable configuration and the corresponding probability: {bitstr, prob}")
    qmine.plot_mine_state(bitstr)
    print("Current profit:               ", qmine.get_profit(bitstr))
    print("Current number of violations: ", qmine.get_violation(bitstr))
    print(x)
    print("------------------------------------------------------------")

In [9]:
#from qiskit.aqua.algorithms import QAOA, VQE
from qiskit.aqua.components.optimizers import COBYLA

p = 4
qaoa = QAOA(qmine.gen_Hamiltonian(penalty=penalty), optimizer=COBYLA(), quantum_instance=instance, p=p)
qaoa.run()
circuit = qaoa.get_optimal_circuit()

analysis(circuit)
# print(circuit.draw())
# print(circuit.decompose().draw())

/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/aqua/components/optimizers/optimizer.py:49: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.components.optimizers',
/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/aqua/algorithms/vq_algorithm.py:70: DeprecationWarning: The class qiskit.aqua.algorithms.VQAlgorithm is deprecated. It was moved/refactored to qiskit.algorithms.VariationalAlgorithm (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_class('aqua.algorithms.VQAlgorithm',
/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/aqua/components/variational_forms/variational_form.py:48: DeprecationWarning: The package q

In [11]:
p = 4

QAOA(qmine.gen_Hamiltonian(penalty=penalty), optimizer=COBYLA(), quantum_instance=instance, p=p).run()

qaoa = QAOA(qmine.gen_Hamiltonian(penalty=penalty), optimizer=COBYLA(), quantum_instance=instance, p=p)
qaoa.run()
circuit = qaoa.get_optimal_circuit()

analysis(circuit)

Average Profit:                           6.324079217188776
Average Number of Smoothness Violations:  0.0011014139604891338
The most probable configuration and the corresponding probability: ('1111', 0.8896484375)
+---+---+---+---+
|   | 0 | 1 | 2 |
+---+---+---+---+
| 0 | 1 | 1 | 1 |
| 1 | x | 1 | x |
+---+---+---+---+
Current profit:                7.0
Current number of violations:  0
{'1101': 0.0048828125, '1001': 0.0009765625, '0011': 0.00390625, '1111': 0.8896484375, '0111': 0.037109375, '0110': 0.0048828125, '0101': 0.05859375}
------------------------------------------------------------


In [10]:
evol_time = 20
nsteps = 50


circuit = ASP(qmine.gen_Hamiltonian(penalty=penalty), 
    evol_time=evol_time, 
    nsteps=nsteps, 
    callback=None, 
    quantum_instance=instance).construct_circuit()

analysis(circuit)

/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/aqua/operators/legacy/weighted_pauli_operator.py:906: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  qc = QuantumCircuit() + state_in
/Users/hxx/.conda/envs/qiskit/lib/python3.9/site-packages/qiskit/circuit/quantumcircuit.py:869: DeprecationWarning: The QuantumCircuit.combine() method is being deprecated. Use the compose() method which is more flexible w.r.t circuit register compatibility.
  return self.combine(rhs)
Average Profit:                           6.753832133010106
Average Number of Smoothness Violations:  8.87761596793435e-05
The most probable configuration and the corresponding probability: ('1111', 0.921875)
+---+---+---+---+
|   | 0 | 1 | 2 |
+---+---+---+---+
| 0 | 1 | 1 | 1 |
| 1 | x | 1 | x |
+---+---+---+---+
Current profit:                7.0
Current number of violations:  0
{'0100': 0.00